In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from pathlib import Path
from ms_pred.common.plot_utils import *
set_style()

/home/samlg/.conda/envs/ms-gen/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset_names = ["nist20", "canopus_train_public"]
split_names = ["split_1.tsv", "scaffold_1.tsv"]
name_to_df = {}
for dataset_name in dataset_names:
    data_folder = Path(f"../data/spec_datasets/{dataset_name}/")
    labels = data_folder / "labels.tsv"
    df = pd.read_csv(labels, sep="\t")
    name_to_df[dataset_name] = df
    # print len of df  in nice f string
    print(f"Len of {dataset_name}: {df.shape[0]:,}")

    # get the intersection of the inchikeys
    grouped_ikeys = df.groupby("inchikey").count()
    # Print shape
    print(f"Len of {dataset_name} after inchikey grouping: {grouped_ikeys.shape[0]:,}")
    # Get intersection of formula
    grouped_formula = df.groupby("formula").count()
    # Print shape
    print(f"Len of {dataset_name} after formula grouping: {grouped_formula.shape[0]:,}")
    
    # Get intersection of ionization
    grouped_ions = df.groupby("ionization").count()
    # Print shape
    print(f"Len of {dataset_name} after ionization grouping: {grouped_ions.shape[0]:,}")
    print(f"{grouped_ions.index}")

    # Print split details
    for split_name in split_names:
        split = data_folder / f"splits/{split_name}"
        if split.exists():
            split_df = pd.read_csv(split, sep="\t")
            split_ct = split_df.groupby("Fold_0").count()
            # Pretty print  the full ct df
            print(f"Split details for {dataset_name} {split_name}")

            print(split_ct)


Len of nist20: 35,129
Len of nist20 after inchikey grouping: 24,403
Len of nist20 after formula grouping: 12,975
Len of nist20 after ionization grouping: 6
Index(['[M+H-2H2O]+', '[M+H-H2O]+', '[M+H]+', '[M+K]+', '[M+NH4]+', '[M+Na]+'], dtype='object', name='ionization')
Split details for nist20 split_1.tsv
         spec
Fold_0       
test     3544
train   28420
val      3165
Split details for nist20 scaffold_1.tsv
         spec
Fold_0       
test     3512
train   28105
val      3512
Len of canopus_train_public: 10,709
Len of canopus_train_public after inchikey grouping: 8,553
Len of canopus_train_public after formula grouping: 5,433
Len of canopus_train_public after ionization grouping: 7
Index(['[M+H3N+H]+', '[M+H]+', '[M+K]+', '[M+Na]+', '[M-H2O+H]+',
       '[M-H4O2+H]+', '[M]+'],
      dtype='object', name='ionization')
Split details for canopus_train_public split_1.tsv
        spec
Fold_0      
test    1040
train   8680
val      989


Saved output:

```
Len of nist20: 35,129
Len of nist20 after inchikey grouping: 24,403
Len of nist20 after formula grouping: 12,975
Len of nist20 after ionization grouping: 6
Index(['[M+H-2H2O]+', '[M+H-H2O]+', '[M+H]+', '[M+K]+', '[M+NH4]+', '[M+Na]+'], dtype='object', name='ionization')
Split details for nist20 split_1.tsv
         spec
Fold_0       
test     3544
train   28420
val      3165
Split details for nist20 scaffold_1.tsv
         spec
Fold_0       
test     3512
train   28105
val      3512
Len of canopus_train_public: 10,709
Len of canopus_train_public after inchikey grouping: 8,553
Len of canopus_train_public after formula grouping: 5,433
Len of canopus_train_public after ionization grouping: 7
Index(['[M+H3N+H]+', '[M+H]+', '[M+K]+', '[M+Na]+', '[M-H2O+H]+',
       '[M-H4O2+H]+', '[M]+'],
      dtype='object', name='ionization')
Split details for canopus_train_public split_1.tsv
        spec
Fold_0      
test    1040
train   8680
val      989
```
